In [1]:
from kaggle_secrets import UserSecretsClient
secret_label = "Token"
token = UserSecretsClient().get_secret(secret_label)

! git clone https://{token}@github.com/CarloSgaravatti/Recsys_Challenge_2023.git

Cloning into 'Recsys_Challenge_2023'...
remote: Enumerating objects: 2234, done.
remote: Counting objects: 100% (352/352), done.
remote: Compressing objects: 100% (208/208), done.
remote: Total 2234 (delta 271), reused 184 (delta 137), pack-reused 1882
Receiving objects: 100% (2234/2234), 378.17 MiB | 31.37 MiB/s, done.
Resolving deltas: 100% (1723/1723), done.


In [2]:
! pip install PyGithub requests

  Obtaining dependency information for PyGithub from https://files.pythonhosted.org/packages/be/04/810d131be173cba445d3658a45512b2b2b3d0960d52c4a300d6ec5e00f52/PyGithub-2.1.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 23.2 MB/s eta 0:00:00


In [3]:
! cd /kaggle/working/Recsys_Challenge_2023 && python run_compile_all_cython.py

run_compile_all_cython: Found 10 Cython files in 4 folders...
run_compile_all_cython: All files will be compiled using your current python environment: '/opt/conda/bin/python'
Compiling [1/10]: MatrixFactorizationImpressions_Cython_Epoch.pyx... 
In file included from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarraytypes.h:1940,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/arrayobject.h:5,
                 from MatrixFactorizationImpressions_Cython_Epoch.c:1109:
/opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^~~~~~~
MatrixFactorizationImpressions_Cython_Epoch.c: In functio

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import json
import random
import tqdm
import gc
import time
import os
import shutil
import optuna
import scipy.sparse as sps
from concurrent.futures import ThreadPoolExecutor

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/recommender-system-2023-challenge-polimi/data_train.csv
/kaggle/input/recommender-system-2023-challenge-polimi/data_target_users_test.csv
/kaggle/input/training-tuned-models/__results__.html
/kaggle/input/training-tuned-models/ease_r_all.zip
/kaggle/input/training-tuned-models/slim_elastic_train_validation.zip
/kaggle/input/training-tuned-models/ease_r_train_validation.zip
/kaggle/input/training-tuned-models/__notebook__.ipynb
/kaggle/input/training-tuned-models/ials_xgboost_train_validation.zip
/kaggle/input/training-tuned-models/ials_train_validation.zip
/kaggle/input/training-tuned-models/slim_elastic_xgboost_train_validation.zip
/kaggle/input/training-tuned-models/ease_r_xgboost_train_validation.zip
/kaggle/input/training-tuned-models/__output__.json
/kaggle/input/training-tuned-models/ials_all.zip
/kaggle/input/training-tuned-models/slim_elastic_all.zip
/kaggle/input/training-tuned-models/custom.css
/kaggle/input/training-tuned-models/Recsys_Challenge_2023/run_all_al

In [5]:
%cd /kaggle/working/Recsys_Challenge_2023

/kaggle/working/Recsys_Challenge_2023


In [6]:
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender, SLIMElasticNetRecommender
from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
from Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender, PureSVDItemRecommender, ScaledPureSVDRecommender
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Recommenders.KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender
from Recommenders.KNN.ItemKNNCustomSimilarityRecommender import ItemKNNCustomSimilarityRecommender
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender
from Recommenders.Neural.MultVAERecommender import MultVAERecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.SLIM.SLIM_BPR_Python import SLIM_BPR_Python
from Recommenders.MatrixFactorization.NMFRecommender import NMFRecommender
from Recommenders.BaseRecommender import BaseRecommender

from Recommenders.Hybrid.LinearCombinationRecommender import LinearCombinationRecommender
from Data_manager.CrossValidationSplitter import CrossValidationSplitter

np.int = int
np.random.seed(42)
np.bool = bool
np.float = float

In [7]:
from github import Github
from github import Auth

auth = Auth.Token(token)
g = Github(auth=auth)

repo = None
for r in g.get_user().get_repos():
    if r.name == 'Recsys_Challenge_2023':
        repo = r
        print('Repository found')

Repository found


In [8]:
def upload_file(filepath_kaggle, filepath_github, commit_message):
    try:
        contents = repo.get_contents(filepath_github)
        with open(filepath_kaggle, "rb") as file:
            repo.update_file(contents.path, commit_message, file.read(), contents.sha)
    except Exception:
        with open(filepath_kaggle, "rb") as file:
            repo.create_file(filepath_github, commit_message, file.read())

In [9]:
config = {
    'tune_parameters': True,
    'database_path': '/kaggle/working/tuning_hybrid_linear_comb.db',
    'copy_prev_tuning_db': True,
    'copy_prev_best_params': True,
    'save_github': True
}

In [10]:
train_data = pd.read_csv('/kaggle/input/recommender-system-2023-challenge-polimi/data_train.csv')
user_ids_test = pd.read_csv('/kaggle/input/recommender-system-2023-challenge-polimi/data_target_users_test.csv')

In [11]:
URM_all = sps.csr_matrix((train_data.data.values,
                          ((train_data.row - 1).values, (train_data.col - 1).values)),
                        shape=(train_data.row.max(), train_data.col.max()))
URM_all

<13024x22347 sparse matrix of type '<class 'numpy.float64'>'
	with 478730 stored elements in Compressed Sparse Row format>

In [12]:
URM_train, URM_test_complete = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)
URM_validation, URM_test = split_train_in_two_percentage_global_sample(URM_test_complete, train_percentage = 0.8)

In [13]:
with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_knn_item.json', 'r') as params_file:
    params_item_knn = json.load(params_file)
params_item_knn

{'topK': 14,
 'shrink': 25,
 'similarity': 'tversky',
 'tversky_alpha': 0.18163095712123836,
 'tversky_beta': 1.8529812241698465}

In [14]:
with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_RP3.json', 'r') as params_file:
    params_rp3 = json.load(params_file)
params_rp3

{'alpha': 0.3408516511081212, 'beta': 0.1447198697205665, 'topK': 32}

In [15]:
with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_knn_user.json', 'r') as params_file:
    params_user_knn = json.load(params_file)
params_user_knn

{'topK': 498,
 'shrink': 0,
 'similarity': 'asymmetric',
 'asymmetric_alpha': 0.39792525383867455}

In [16]:
with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_alpha.json', 'r') as params_file:
    params_p3alpha = json.load(params_file)
params_p3alpha

{'normalize_similarity': 'True',
 'implicit': 'True',
 'alpha': 0.29354071531607107,
 'topK': 49}

In [17]:
with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_hybrid_merging.json', 'r') as params_file:
    params_merging = json.load(params_file)
params_merging

{'alpha': 0.3375591188839621, 'topK': 86}

In [18]:
with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/pure_svd_params.json', 'r') as params_file:
    params_pure_svd = json.load(params_file)
params_pure_svd

{'factors': 30}

In [19]:
# with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_puresvd_item.json', 'r') as params_file:
#     params_pure_svd_item = json.load(params_file)
params_pure_svd_item = {"topK": 43, "num_factors": 171}

In [20]:
with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_puresvd_scaled.json', 'r') as params_file:
    params_scaled = json.load(params_file)
params_scaled

{'num_factors': 179,
 'scaling_items': 5.3298425368534826e-05,
 'scaling_users': 0.2791990262721904}

In [21]:
shutil.copyfile(
    '/kaggle/input/training-tuned-models/ials_train_validation.zip', 
    '/kaggle/working/ials_train_validation.zip', 
)
shutil.copyfile(
    '/kaggle/input/training-tuned-models/ials_train_validation.zip', 
    '/kaggle/working/ials_all.zip', 
)

'/kaggle/working/ials_all.zip'

In [22]:
recommender_item = ItemKNNCFRecommender(URM_train)
recommender_item.fit(**params_item_knn)

recommender_user = UserKNNCFRecommender(URM_train)
recommender_user.fit(**params_user_knn)

recommender_rp3 = RP3betaRecommender(URM_train)
recommender_rp3.fit(**params_rp3)

recommender_p3a = P3alphaRecommender(URM_train)
recommender_p3a.fit(**params_p3alpha)

recommender_pure_svd = PureSVDRecommender(URM_train)
recommender_pure_svd.fit(num_factors=params_pure_svd['factors'])

recommender_pure_svd_item = PureSVDItemRecommender(URM_train)
recommender_pure_svd_item.fit(**params_pure_svd_item)

recommender_scaled_svd = ScaledPureSVDRecommender(URM_train)
recommender_scaled_svd.fit(**params_scaled) 

recommender_nmf = NMFRecommender(URM_train)
recommender_nmf.load_model('/kaggle/working/Recsys_Challenge_2023/models/', 'nmf_train_validation.zip')

recommender_slim_bpr = SLIM_BPR_Python(URM_train)
recommender_slim_bpr.load_model('/kaggle/working/Recsys_Challenge_2023/models/', 'slim_bpr_train_validation.zip')

recommender_slim = SLIMElasticNetRecommender(URM_train)
recommender_slim.load_model('/kaggle/working/Recsys_Challenge_2023/models/', 'slim_elastic_train_validation.zip')

recommender_ease_r = EASE_R_Recommender(URM_train)
recommender_ease_r.load_model('/kaggle/working/Recsys_Challenge_2023/models/', 'ease_r_train_validation.zip')

recommender_ials = IALSRecommender(URM_train)
recommender_ials.load_model('/kaggle/working/', 'ials_train_validation.zip')

hybrid_similarity = ItemKNNSimilarityHybridRecommender(URM_train, recommender_item.W_sparse, recommender_rp3.W_sparse)
hybrid_similarity.fit(**params_merging)

ItemKNNCFRecommender: URM Detected 627 ( 4.8%) users with no interactions.
ItemKNNCFRecommender: URM Detected 247 ( 1.1%) items with no interactions.
Similarity column 22347 (100.0%), 6514.72 column/sec. Elapsed time 3.43 sec
UserKNNCFRecommender: URM Detected 627 ( 4.8%) users with no interactions.
UserKNNCFRecommender: URM Detected 247 ( 1.1%) items with no interactions.
Similarity column 13024 (100.0%), 5661.99 column/sec. Elapsed time 2.30 sec
RP3betaRecommender: URM Detected 627 ( 4.8%) users with no interactions.
RP3betaRecommender: URM Detected 247 ( 1.1%) items with no interactions.
RP3betaRecommender: Similarity column 22347 (100.0%), 4251.95 column/sec. Elapsed time 5.26 sec
P3alphaRecommender: URM Detected 627 ( 4.8%) users with no interactions.
P3alphaRecommender: URM Detected 247 ( 1.1%) items with no interactions.
P3alphaRecommender: Similarity column 22347 (100.0%), 4309.84 column/sec. Elapsed time 5.19 sec
PureSVDRecommender: URM Detected 627 ( 4.8%) users with no inter

In [23]:
weights = [0.4, 0.4, 0.2]
linear_combinator = LinearCombinationRecommender(URM_train, [recommender_item, recommender_rp3, recommender_p3a], weights)
evaluator = EvaluatorHoldout(URM_test_complete, cutoff_list=[10])

print('Linear Combinator (Item knn, p3alpha, rp3)')
result_df, _ = evaluator.evaluateRecommender(linear_combinator)
display(result_df)
print('Merging Models')
result_df, _ = evaluator.evaluateRecommender(hybrid_similarity)
display(result_df)
print('Item KNN')
result_df, _ = evaluator.evaluateRecommender(recommender_item)
display(result_df)
print('RP3 beta')
result_df, _ = evaluator.evaluateRecommender(recommender_rp3)
display(result_df)
print('P3 alpha')
result_df, _ = evaluator.evaluateRecommender(recommender_p3a)
display(result_df)
print('User KNN')
result_df, _ = evaluator.evaluateRecommender(recommender_user)
display(result_df)
print('NMF')
result_df, _ = evaluator.evaluateRecommender(recommender_nmf)
display(result_df)
print('Pure SVD')
result_df, _ = evaluator.evaluateRecommender(recommender_pure_svd)
display(result_df)
print('Scaled Pure SVD')
result_df, _ = evaluator.evaluateRecommender(recommender_scaled_svd)
display(result_df)
print('Item Pure SVD')
result_df, _ = evaluator.evaluateRecommender(recommender_pure_svd_item)
display(result_df)
print('SLIM BPR')
result_df, _ = evaluator.evaluateRecommender(recommender_slim_bpr)
display(result_df)
print('SLIM Elastic')
result_df, _ = evaluator.evaluateRecommender(recommender_slim)
display(result_df)
print('EASE R')
result_df, _ = evaluator.evaluateRecommender(recommender_ease_r)
display(result_df)
print('IALS')
result_df, _ = evaluator.evaluateRecommender(recommender_ials)
display(result_df)

Recommender_Base_Class: URM Detected 627 ( 4.8%) users with no interactions.
Recommender_Base_Class: URM Detected 247 ( 1.1%) items with no interactions.
EvaluatorHoldout: Ignoring 2568 (19.7%) Users that have less than 1 test interactions
Linear Combinator (Item knn, p3alpha, rp3)
EvaluatorHoldout: Processed 10456 (100.0%) in 20.69 sec. Users per second: 505


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.096538,0.165157,0.138974,0.049938,0.085155,0.27383,0.152219,0.113933,0.517215,0.36479,...,0.802826,0.415233,0.802826,0.079362,10.588395,0.998316,0.22899,0.815689,1.562376,0.307388


Merging Models
EvaluatorHoldout: Processed 10456 (100.0%) in 10.53 sec. Users per second: 993


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.096595,0.165167,0.138965,0.050024,0.085384,0.274834,0.152525,0.11397,0.519702,0.365722,...,0.802826,0.41723,0.802826,0.083179,10.657938,0.998402,0.240004,0.821046,1.518592,0.309028


Item KNN
EvaluatorHoldout: Processed 10456 (100.0%) in 10.39 sec. Users per second: 1006


...

,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.097752,0.165757,0.139187,0.05124,0.086426,0.282607,0.154679,0.114847,0.52477,0.375707,...,0.802826,0.421299,0.802826,0.059374,10.474793,0.998673,0.171318,0.806937,1.547674,0.300111


EASE R
EvaluatorHoldout: Processed 10456 (100.0%) in 9.18 sec. Users per second: 1139


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.08202,0.144797,0.123181,0.0415,0.07293,0.247944,0.132891,0.098472,0.480011,0.318015,...,0.802826,0.385365,0.802826,0.02437,8.98298,0.995273,0.070318,0.692014,2.489147,0.276668


IALS
EvaluatorHoldout: Processed 10456 (100.0%) in 10.79 sec. Users per second: 969


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.08224,0.144123,0.122645,0.041501,0.071584,0.246255,0.131701,0.098458,0.479055,0.316769,...,0.802826,0.384598,0.802826,0.029074,9.74689,0.998532,0.08389,0.750862,1.599865,0.289534


In [24]:
weights = [0.8, 0.2, 0.5, 0.3, 0.1, 0.1, 0.01, 0.005, 0.005, 0.005, 0.002]
recommenders = [recommender_slim, recommender_item, recommender_rp3, recommender_p3a, recommender_ials, recommender_ease_r, recommender_user, recommender_pure_svd_item, recommender_scaled_svd, recommender_nmf, recommender_slim_bpr]
linear_combinator = LinearCombinationRecommender(URM_train, recommenders, weights)
evaluator = EvaluatorHoldout(URM_test_complete, cutoff_list=[10])
result_df, _ = evaluator.evaluateRecommender(linear_combinator)
display(result_df)

Recommender_Base_Class: URM Detected 627 ( 4.8%) users with no interactions.
Recommender_Base_Class: URM Detected 247 ( 1.1%) items with no interactions.
EvaluatorHoldout: Ignoring 2568 (19.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 10456 (100.0%) in 36.61 sec. Users per second: 286


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.099962,0.172375,0.14548,0.051917,0.088516,0.284203,0.158242,0.1185,0.535004,0.379147,...,0.802826,0.429515,0.802826,0.055971,10.188449,0.997782,0.161499,0.784878,1.832264,0.296225


In [25]:
evaluator = EvaluatorHoldout(URM_test_complete, cutoff_list=[10])

def objective(trial):
    weights = [
        trial.suggest_float('slim_elastic', 1e-2, 1, log=True),
        trial.suggest_float('item_knn', 1e-2, 1, log=True),
        trial.suggest_float('rp3', 1e-2, 1, log=True),
        trial.suggest_float('p3alpha', 1e-2, 1, log=True),
        trial.suggest_float('ials', 5e-3, 1, log=True),
        trial.suggest_float('ease_r', 1e-3, 1, log=True),
        trial.suggest_float('user_knn', 1e-3, 1, log=True),
        trial.suggest_float('svd_item', 1e-4, 0.2, log=True),
        trial.suggest_float('scaled_svd', 1e-4, 0.2, log=True),
        trial.suggest_float('nmf', 1e-4, 0.2, log=True),
        trial.suggest_float('slim_bpr', 1e-5, 0.1, log=True),
    ]
    
    recommenders = [
        recommender_slim, recommender_item, recommender_rp3, recommender_p3a, recommender_ials, recommender_ease_r, 
        recommender_user, recommender_pure_svd_item, recommender_scaled_svd, recommender_nmf, recommender_slim_bpr
    ]
    
    linear_combinator = LinearCombinationRecommender(URM_train, recommenders, weights)
    evaluator = EvaluatorHoldout(URM_test_complete, cutoff_list=[10])
    result_df, _ = evaluator.evaluateRecommender(linear_combinator)

    return result_df.loc[10, 'MAP']

EvaluatorHoldout: Ignoring 2568 (19.7%) Users that have less than 1 test interactions


In [26]:
try:
    shutil.copyfile(
        '/kaggle/working/Recsys_Challenge_2023/tuning_results/tuning_hybrid_merging.db', 
        config['database_path']
    )
except FileNotFoundError:
    pass

In [27]:
if config['tune_parameters']:
    study = optuna.create_study(direction='maximize', study_name='hyperparameters_tuning_linear_11models_no_cv', 
                                storage=f'sqlite:///{config["database_path"]}', load_if_exists=True)
    study.optimize(objective, n_trials=200)

[I 2023-12-05 14:20:06,727] A new study created in RDB with name: hyperparameters_tuning_linear_11models_no_cv


Recommender_Base_Class: URM Detected 627 ( 4.8%) users with no interactions.
Recommender_Base_Class: URM Detected 247 ( 1.1%) items with no interactions.
EvaluatorHoldout: Ignoring 2568 (19.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 10456 (100.0%) in 36.89 sec. Users per second: 283


[I 2023-12-05 14:20:43,949] Trial 0 finished with value: 0.05109123188933909 and parameters: {'slim_elastic': 0.4190258028154633, 'item_knn': 0.45515710880672955, 'rp3': 0.07658491356966007, 'p3alpha': 0.37522622318903787, 'ials': 0.09313648438995077, 'ease_r': 0.004630842906408421, 'user_knn': 0.013361800872920312, 'svd_item': 0.0003220597875883612, 'scaled_svd': 0.004465566412666983, 'nmf': 0.0004872489959520195, 'slim_bpr': 8.142214259202564e-05}. Best is trial 0 with value: 0.05109123188933909.


Recommender_Base_Class: URM Detected 627 ( 4.8%) users with no interactions.
Recommender_Base_Class: URM Detected 247 ( 1.1%) items with no interactions.
EvaluatorHoldout: Ignoring 2568 (19.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 10456 (100.0%) in 36.64 sec. Users per second: 285


[I 2023-12-05 14:21:20,864] Trial 1 finished with value: 0.049198280625691315 and parameters: {'slim_elastic': 0.07709300009697584, 'item_knn': 0.03214676102701574, 'rp3': 0.29655542781905025, 'p3alpha': 0.016853471912978153, 'ials': 0.18273757168891666, 'ease_r': 0.35854394082760926, 'user_knn': 0.0014605855240286598, 'svd_item': 0.03641480672157251, 'scaled_svd': 0.019632831120780497, 'nmf': 0.0010708276366613004, 'slim_bpr': 0.0016090050646071989}. Best is trial 0 with value: 0.05109123188933909.


...

[I 2023-12-05 16:25:59,139] Trial 197 finished with value: 0.05159892626395175 and parameters: {'slim_elastic': 0.18259241918422695, 'item_knn': 0.06407331070285553, 'rp3': 0.04281349653001735, 'p3alpha': 0.25670361668757075, 'ials': 0.005132696523679739, 'ease_r': 0.0039673856045952536, 'user_knn': 0.0023987914579571765, 'svd_item': 0.00026034329952739563, 'scaled_svd': 0.0022911933782194874, 'nmf': 0.0027093833929485306, 'slim_bpr': 0.0010895318834296631}. Best is trial 89 with value: 0.05225166912595235.


Recommender_Base_Class: URM Detected 627 ( 4.8%) users with no interactions.
Recommender_Base_Class: URM Detected 247 ( 1.1%) items with no interactions.
EvaluatorHoldout: Ignoring 2568 (19.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 10456 (100.0%) in 39.57 sec. Users per second: 264


[I 2023-12-05 16:26:39,131] Trial 198 finished with value: 0.0522029388457759 and parameters: {'slim_elastic': 0.799996971468844, 'item_knn': 0.048034878483072425, 'rp3': 0.04780049878580867, 'p3alpha': 0.29881601220798415, 'ials': 0.005016384349352227, 'ease_r': 0.0044476306772693415, 'user_knn': 0.002625443479717226, 'svd_item': 0.00023544254776256454, 'scaled_svd': 0.0010007318990232605, 'nmf': 0.002189800888608887, 'slim_bpr': 0.0012893769598391028}. Best is trial 89 with value: 0.05225166912595235.


Recommender_Base_Class: URM Detected 627 ( 4.8%) users with no interactions.
Recommender_Base_Class: URM Detected 247 ( 1.1%) items with no interactions.
EvaluatorHoldout: Ignoring 2568 (19.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 10456 (100.0%) in 39.37 sec. Users per second: 266


[I 2023-12-05 16:27:18,918] Trial 199 finished with value: 0.051868696184161484 and parameters: {'slim_elastic': 0.6205463212619581, 'item_knn': 0.05989969369884237, 'rp3': 0.037067999547439584, 'p3alpha': 0.6550301237836478, 'ials': 0.006357572551805945, 'ease_r': 0.0030807802632774623, 'user_knn': 0.003947064623261249, 'svd_item': 0.0002083683672114649, 'scaled_svd': 0.00015075368901217297, 'nmf': 0.00327877252119434, 'slim_bpr': 0.0009208285282328525}. Best is trial 89 with value: 0.05225166912595235.


In [28]:
if config['tune_parameters']:
    fig = optuna.visualization.plot_optimization_history(study)
    fig.show()

In [29]:
if config['tune_parameters']:
    fig = optuna.visualization.plot_param_importances(study)
    fig.show()

In [30]:
if config['tune_parameters']:
    fig = optuna.visualization.plot_slice(study)
    fig.show()

In [31]:
if config['tune_parameters']:
    with open('/kaggle/working/best_params_hybrid_lr_comb_11models.json', 'w') as params_file:
        json.dump(study.best_params, params_file)
        
    if config['save_github']: 
        upload_file(
            '/kaggle/working/best_params_hybrid_lr_comb_11models.json', 
            'tuning_results/best_params_hybrid_lr_comb_11models.json', 
            'Linear combination of 11 models tuning results (from kaggle notebook)'
        )
    
elif config['copy_prev_best_params']:
    shutil.copyfile(
        '/kaggle/input/linear-combination/best_params_hybrid_lr_comb_11models.json', 
        '/kaggle/working/best_params_hybrid_lr_comb_11models.json'
    )

In [32]:
if config['save_github'] and config['tune_parameters']:
    upload_file(
        config['database_path'],
        'tuning_results/tuning_hybrid_merging.db', 
        'Linear combination tuning db updated results (from kaggle notebook)'
    )

In [33]:
with open('/kaggle/working/best_params_hybrid_lr_comb_11models.json', 'r') as params_file:
    params = json.load(params_file)
params

{'slim_elastic': 0.781237039707711,
 'item_knn': 0.034069197683945486,
 'rp3': 0.036876076567740015,
 'p3alpha': 0.4828549014067903,
 'ials': 0.005962153700988408,
 'ease_r': 0.0049438034490025635,
 'user_knn': 0.006035100932659037,
 'svd_item': 0.0004310311961633854,
 'scaled_svd': 0.00012719243110155445,
 'nmf': 0.003321344691440341,
 'slim_bpr': 0.0001851746449226467}

In [34]:
recommender_item = ItemKNNCFRecommender(URM_all)
recommender_item.fit(**params_item_knn)

recommender_user = UserKNNCFRecommender(URM_all)
recommender_user.fit(**params_user_knn)

recommender_rp3 = RP3betaRecommender(URM_all)
recommender_rp3.fit(**params_rp3)

recommender_p3a = P3alphaRecommender(URM_all)
recommender_p3a.fit(**params_p3alpha)

recommender_pure_svd = PureSVDRecommender(URM_all)
recommender_pure_svd.fit(num_factors=params_pure_svd['factors'])

recommender_pure_svd_item = PureSVDItemRecommender(URM_all)
recommender_pure_svd_item.fit(**params_pure_svd_item)

recommender_scaled_svd = ScaledPureSVDRecommender(URM_all)
recommender_scaled_svd.fit(**params_scaled) 

recommender_nmf = NMFRecommender(URM_all)
recommender_nmf.load_model('/kaggle/working/Recsys_Challenge_2023/models/', 'nmf_all.zip')

recommender_slim_bpr = SLIM_BPR_Python(URM_all)
recommender_slim_bpr.load_model('/kaggle/working/Recsys_Challenge_2023/models/', 'slim_bpr_all.zip')

recommender_slim = SLIMElasticNetRecommender(URM_all)
recommender_slim.load_model('/kaggle/working/Recsys_Challenge_2023/models/', 'slim_elastic_all.zip')

recommender_ease_r = EASE_R_Recommender(URM_all)
recommender_ease_r.load_model('/kaggle/working/Recsys_Challenge_2023/models/', 'ease_r_all.zip')

recommender_ials = IALSRecommender(URM_all)
recommender_ials.load_model('/kaggle/working/', 'ials_all.zip')

ItemKNNCFRecommender: URM Detected 386 ( 3.0%) users with no interactions.
ItemKNNCFRecommender: URM Detected 125 ( 0.6%) items with no interactions.
Similarity column 22347 (100.0%), 5433.39 column/sec. Elapsed time 4.11 sec
UserKNNCFRecommender: URM Detected 386 ( 3.0%) users with no interactions.
UserKNNCFRecommender: URM Detected 125 ( 0.6%) items with no interactions.
Similarity column 13024 (100.0%), 4276.77 column/sec. Elapsed time 3.05 sec
RP3betaRecommender: URM Detected 386 ( 3.0%) users with no interactions.
RP3betaRecommender: URM Detected 125 ( 0.6%) items with no interactions.
RP3betaRecommender: Similarity column 22347 (100.0%), 3399.51 column/sec. Elapsed time 6.57 sec
P3alphaRecommender: URM Detected 386 ( 3.0%) users with no interactions.
P3alphaRecommender: URM Detected 125 ( 0.6%) items with no interactions.
P3alphaRecommender: Similarity column 22347 (100.0%), 3504.40 column/sec. Elapsed time 6.38 sec
PureSVDRecommender: URM Detected 386 ( 3.0%) users with no inter

In [35]:
weights = [
    params['slim_elastic'],
    params['item_knn'],
    params['rp3'],
    params['p3alpha'],
    params['ials'],
    params['ease_r'],
    params['user_knn'],
    params['svd_item'],
    params['scaled_svd'],
    params['nmf'],
    params['slim_bpr']
]

recommenders = [
    recommender_slim, recommender_item, recommender_rp3, recommender_p3a, recommender_ials, recommender_ease_r,
    recommender_user, recommender_pure_svd_item, recommender_scaled_svd, recommender_nmf, recommender_slim_bpr
]

In [36]:
linear_combinator = LinearCombinationRecommender(URM_all, recommenders, weights)
recommendations = linear_combinator.recommend(user_ids_test['user_id'].unique() - 1, cutoff=10, return_scores=False)
recommendations = [' '.join([str(item + 1) for item in r]).strip() for r in recommendations]

submission_df = pd.DataFrame({'user_id': user_ids_test['user_id'].unique(), 'item_list': recommendations})
display(submission_df.head())

submission_df.to_csv('/kaggle/working/submission_linear_comb_11models.csv', index=False)

Recommender_Base_Class: URM Detected 386 ( 3.0%) users with no interactions.
Recommender_Base_Class: URM Detected 125 ( 0.6%) items with no interactions.


,user_id,item_list
0,1,36 101 123 506 403 515 694 1546 52 1422
1,2,1095 47 12 1522 50 28 656 196 11 949
2,3,59 857 4252 259 2172 648 4 536 1281 584
3,4,249 28 50 139 171 314 146 254 7 5
4,5,1570 170 77 95 131 1511 471 5138 1220 116
